# 6. 데이터 로딩과 저장, 파일 형식
**"데이터는 텍스트 데이터와 바이너리 데이터로 구분 가능"**

## 6.1 텍스트 데이터
**"사람이 읽기 편한 형식"**

## 6.2 바이너리 데이터
**"사람이 읽기 어려운 형식"**

- 파이썬에서는 기본적으로 pickle을 이용해 바이너리 데이터를 다룸
- **HDF5, 엑셀,** Message-Pack, Bcolz, Feather 등도 지원

In [1]:
import numpy as np
import pandas as pd

In [2]:
frame = pd.read_csv('examples/ex1.csv') # 텍스트 데이터 읽기
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [3]:
frame.to_pickle('examples/frame_pickle') # 바이너리 데이터로 저장

In [4]:
pd.read_pickle('examples/frame_pickle') # 바이너리 데이터 읽기

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


### 6.2.1 HDF5 형식

- Hierachical Data Format(계층적 데이터 형식)
- **대량의 과학 계산용** 배열 데이터를 저장하는데 유용
- on_the_fly(실시간 압축)을 지원. **반복되는 패턴을 가진 데이터** 저장 시 유용
- 메모리에 모두 적재할 수 없는 큰 데이터에서 **필요한 부분만** 효과적으로 읽고쓰기 가능

In [5]:
frame = pd.DataFrame({'a': np.random.randn(100)}) # 표준정규분포에서 100개의 실수 생성
store = pd.HDFStore('examples/mydata.h5') # 저장 경로와 형식 지정
store['obj1'] = frame # 딕셔너리. 키: obj1, 값: frame
store['obj1_col'] = frame['a'] # colums 이름: a
store # HDF5 객체 생성

<class 'pandas.io.pytables.HDFStore'>
File path: examples/mydata.h5

In [6]:
store['obj1']

,a
0,-0.502475
1,0.302246
2,-0.724473
3,0.651024
4,-0.967766
...,...
95,1.671338
96,0.843847
97,0.179694
98,0.245377


In [7]:
# 원하는 방식 사용1
store.open()
store.put('obj2', frame, format='table') # obj2로 복사
store.select('obj2', where=['index >= 96 and index <= 98']) # 범위 선택해서 읽기

,a
96,0.843847
97,0.179694
98,0.245377


In [8]:
store.close()

In [9]:
# 원하는 방식 사용2
frame.to_hdf('mydata.h5', 'obj3', format='table') # obj3으로 복사
pd.read_hdf('mydata.h5', 'obj3', where=['index < 5']) # 범위 선택해서 읽기

,a
0,-0.502475
1,0.302246
2,-0.724473
3,0.651024
4,-0.967766


### 6.2.2 엑셀 형식

#### 엑셀 읽기

In [10]:
# xlrd, openpyxl 패키지 설치

# 원하는 방식 사용1
xlsx = pd.ExcelFile('examples/ex1.xlsx') # 엑셀 객체 생성
pd.read_excel(xlsx, 'Sheet1') # 읽기

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [11]:
# 원하는 방식 사용2
frame = pd.read_excel('examples/ex1.xlsx', 'Sheet1') 
frame # 객체 생성 없이 명령어만으로도 읽기 가능

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


#### 엑셀 쓰기

In [12]:
#원하는 방식 사용 1
writer = pd.ExcelWriter('examples/ex2.xlsx') # 엑셀 객체 생성
frame.to_excel(writer, 'Sheet1') # 쓰기
writer.save() # 저장

In [13]:
#원하는 방식 사용 2
frame.to_excel('examples/ex2.xlsx') # 쓰고 저장하기
# 객체 생성 없이 명령어만으로도 쓰기 가능

## 6-3 웹 API 이용하는 법
**"웹사이트의 데이터 가져오기"**

In [14]:
import requests

url = "https://api.github.com/repos/pandas-dev/pandas/issues"
resp = requests.get(url) # 데이터 요청
resp # 200: 정상 작동

<Response [200]>

**JSON(JavaScript Object Notation):** 웹브라우저와 다른 애플리케이션이 HTTP요청으로 데이터를 보낼 때 널리 사용하는 표준 파일 형식 중 하나

In [15]:
data = resp.json() # 딕셔너리 형태의 객체 반환

issues = pd.DataFrame(data, columns=['number', 'title', 'labels', 'state'])
issues

,number,title,labels,state
0,42777,DOC GH42756 Update documentation for drop and ...,[],open
1,42776,BUG: Incorrect variable window bounds for firs...,[],open
2,42775,fixes docstring in `to_hdf()`,[],open
3,42774,PERF: extract_array earlier in DataFrame const...,[],open
4,42773,CI: mypy fixup,[],open
5,42772,BUG: Series.groupby fails with InvalidIndexErr...,[],open
6,42771,BUG: DataFrame.drop() can't drop row by index ...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
7,42770,Fix typing issues for CI,[],open
8,42769,CI: numpy 1.21.1 breaks mypy validation,"[{'id': 48070600, 'node_id': 'MDU6TGFiZWw0ODA3...",open
9,42768,BUG: to_json() swallows attributes of dataclasses,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open


## 6-4 데이터베이스 이용하는 법
**"SQL(Structured Query Language)기반의 관계형 데이터베이스에서 데이터 가져오기"**

In [20]:
# SQLite는 별도의 DB 서버가 필요없이 DB 파일에 기초하여 데이터베이스 처리를 구현한 Embedded SQL DB 엔진
import sqlite3

# 테이블 생성 / 테이블 이름:test / 값: a(20자 이내) b(20자 이내) c(실제) d(정수형)
query = """
CREATE TABLE test 
(a VARCHAR(20), b VARCHAR(20), c REAL, d INTEGER);
"""

con = sqlite3.connect('mydata.sqlite') # 객체 생성

con.execute(query) # 쿼리 실행(빈 테이블 생성)

In [21]:
con.commit()

In [22]:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]

stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"
con.executemany(stmt, data) # 테이블 채우기 실행(구조, 값)

In [23]:
con.commit()

In [24]:
cursor = con.execute('select * from test') # select문 실행(description 메서드 사용하기 위해)

rows = cursor.fetchall() # row list 가져오기. select문 실행 이후 데이터를 꺼내려면 fetchall() 등 사용해야함

rows # 데이터

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [25]:
cursor.description # colums 이름 가져오기. 파이썬 DB API와의 호환성을 유지하기 위해 각 열마다 7-튜플을 반환(마지막 6개 None)

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [26]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


In [27]:
# sqlalchemy 패키지 설치
import sqlalchemy as sqla

db = sqla.create_engine('sqlite:///mydata.sqlite') # 경로 설정

pd.read_sql('select * from test', db) # 이전에 만들어둔 sql 읽기

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
